In [1]:
from bs4 import BeautifulSoup as bs
import requests 
import pymongo
import pandas as pd
import re
import os
import csv
import time
import json
import ast
import pprint
from datetime import datetime, timedelta

In [2]:
def create_geojson(date):
    df_countries_by_day = df_merged[(df_merged.date==date)]
    df_data = df_countries_by_day.groupby(['country','country_2']).agg({'conf_count':'sum', 'cured_count':'sum', 'dead_count':'sum'}).reset_index()
    df_data = df_data.set_index('country')
    
                                          
    json_df = pd.read_json('static/world_geojsons/world.json', encoding='UTF-8')
    json_df_feat = pd.DataFrame(json_df.features)

    geo_dict = {}
    geo_string = ""

    for index, row in json_df_feat.iterrows():
#         print(row['features']['properties']['ADMIN'])
        try:
            str_feat_1 = ("{\"type\": \"Feature\",")

            prop_name = row['features']['properties']['ADMIN']
            prop_american_name = df_data.loc[prop_name,"country_2"]
            prop_confirmedCount = df_data.loc[prop_name,"conf_count"]
            prop_suspectedCount = 0
            prop_curedCount = df_data.loc[prop_name,"cured_count"]
            prop_deadCount = df_data.loc[prop_name,"dead_count"]    
            prop_date = date

            str_prop_double_quotes = str(row['features']['properties'])
            str_prop_double_quotes = str_prop_double_quotes.replace("\'","\"")

            str_prop_1 = ("\"properties\" : " + str_prop_double_quotes + "\",")[:-3]
            str_prop_2 = (",\"american_name\" : \"" + prop_american_name + "\",")
            str_prop_3 = ("\"confirmedCount\" : \"" + str(prop_confirmedCount) + "\",")
            str_prop_4 = ("\"suspectedCount\" : \"" + str(prop_suspectedCount) + "\",")
            str_prop_5 = ("\"curedCount\" : \"" + str(prop_curedCount) + "\",")
            str_prop_6 = ("\"deadCount\" : \"" + str(prop_deadCount) + "\",")
            str_prop_7 = ("\"date\" : \"" + prop_date + "\"},")

            str_prop_all  = str_prop_1 + str_prop_2 + str_prop_3 + str_prop_4 + str_prop_5 + str_prop_6 + str_prop_7
#             print(str_prop_all)
            str_geom_1 = ("\"geometry\":" + str(row['features']['geometry']) + "},")
            str_geom_1 = str_geom_1.replace("\'","\"")
#             print(str_geom_1)

            str_for_each_province = (str_feat_1)+(str_prop_all)+(str_geom_1)
#             print(str_for_each_province)
            geo_string = geo_string + (str_for_each_province)
        except:
            str_feat_1 = ("{\"type\": \"Feature\",")

            prop_name = row['features']['properties']['ADMIN']

            prop_american_name = ""
            prop_confirmedCount = 0
            prop_suspectedCount = 0
            prop_curedCount = 0
            prop_deadCount = 0  
            prop_date = date

            str_prop_double_quotes = str(row['features']['properties'])
            str_prop_double_quotes = str_prop_double_quotes.replace("\'","\"")

            str_prop_1 = ("\"properties\" : " + str_prop_double_quotes + "\",")[:-3]
            str_prop_2 = (",\"american_name\" : \"" + prop_american_name + "\",")
            str_prop_3 = ("\"confirmedCount\" : \"" + str(prop_confirmedCount) + "\",")
            str_prop_4 = ("\"suspectedCount\" : \"" + str(prop_suspectedCount) + "\",")
            str_prop_5 = ("\"curedCount\" : \"" + str(prop_curedCount) + "\",")
            str_prop_6 = ("\"deadCount\" : \"" + str(prop_deadCount) + "\",")
            str_prop_7 = ("\"date\" : \"" + prop_date + "\"},")

            str_prop_all  = str_prop_1 + str_prop_2 + str_prop_3 + str_prop_4 + str_prop_5 + str_prop_6 + str_prop_7
            
            str_geom_1 = ("\"geometry\":" + str(row['features']['geometry']) + "},")
            str_geom_1 = str_geom_1.replace("\'","\"")

            str_for_each_province = (str_feat_1)+(str_prop_all)+(str_geom_1)
            geo_string = geo_string + (str_for_each_province)
    pre_fix = "{\"type\": \"FeatureCollection\", \"features\": ["
    post_fix = "]}"

    total_string = pre_fix + geo_string[:-1] + post_fix

#     print(total_string)
    output_path = os.path.join("static/world_geojsons", date + ".json")
    with open(output_path, "w", encoding='UTF-8') as text_file:
        text_file.write(total_string)
        text_file.close()

In [3]:
# Get the data from 3 different data source
confirmed_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
df_confirmed = pd.read_csv(confirmed_url)
cured_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'
df_cured = pd.read_csv(cured_url)
death_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
df_death = pd.read_csv(death_url)

df_confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20
0,Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,70,...,986,987,988,989,989,989,989,989,989,990
1,Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,80,...,393,395,396,399,399,399,400,400,410,410
2,Chongqing,Mainland China,30.0572,107.8740,6,9,27,57,75,110,...,560,567,572,573,575,576,576,576,576,576
3,Fujian,Mainland China,26.0789,117.9874,1,5,10,18,35,59,...,293,293,293,293,293,293,294,294,296,296
4,Gansu,Mainland China,36.0611,103.8343,0,2,2,4,7,14,...,91,91,91,91,91,91,91,91,91,91


In [4]:
# Create unique rows by moving date column to row level for all Confirmed cases
df_confirmed_long = df_confirmed.melt(id_vars=["Province/State", "Country/Region","Lat","Long"], 
        var_name="Date", 
        value_name="Value")
df_confirmed_long.head()
output_path = os.path.join("static/world_data", "df_world_confirmed_original.csv")
df_confirmed.to_csv(output_path)

# Create unique rows by moving date column to row level for all Cured cases
df_cured_long = df_cured.melt(id_vars=["Province/State", "Country/Region","Lat","Long"], 
        var_name="Date", 
        value_name="Value")
df_cured_long.head()
output_path = os.path.join("static/world_data", "df_world_cured_original.csv")
df_cured.to_csv(output_path)

# Create unique rows by moving date column to row level for all Death cases
df_death_long = df_death.melt(id_vars=["Province/State", "Country/Region","Lat","Long"], 
        var_name="Date", 
        value_name="Value")
df_death_long.head()
output_path = os.path.join("static/world_data", "df_world_death_original.csv")
df_death.to_csv(output_path)

df_confirmed_long.head()
df_confirmed_long["new_date"] = pd.to_datetime(df_confirmed_long['Date'])
# df_confirmed_long.drop(['Date'], axis=1)

# df_temp=df_death_long[df_death_long["Province/State"].isnull()]
# df_temp.head()
df_confirmed_long.head()


,Province/State,Country/Region,Lat,Long,Date,Value,new_date
0,Anhui,Mainland China,31.8257,117.2264,1/22/20,1,2020-01-22
1,Beijing,Mainland China,40.1824,116.4142,1/22/20,14,2020-01-22
2,Chongqing,Mainland China,30.0572,107.8740,1/22/20,6,2020-01-22
3,Fujian,Mainland China,26.0789,117.9874,1/22/20,1,2020-01-22
4,Gansu,Mainland China,36.0611,103.8343,1/22/20,0,2020-01-22


In [5]:
# If Province/State isn't filled, use the country value instead

df_confirmed_long["Province/State"].fillna(df_confirmed_long["Country/Region"], inplace=True)
df_confirmed_long = df_confirmed_long.rename(columns={"Province/State":"american_name", "Country/Region":"country", "Lat":"lat","Long":"long", "Date":"date", "Value": "conf_count"})

df_cured_long["Province/State"].fillna(df_cured_long["Country/Region"], inplace=True)
df_cured_long = df_cured_long.rename(columns={"Province/State":"american_name", "Country/Region":"country", "Lat":"lat","Long":"long","Date":"date", "Value": "cured_count"})

df_death_long["Province/State"].fillna(df_death_long["Country/Region"], inplace=True)
df_death_long = df_death_long.rename(columns={"Province/State":"american_name", "Country/Region":"country", "Lat":"lat","Long":"long","Date":"date", "Value": "dead_count"})


x = df_confirmed_long[df_confirmed_long['country'] == 'US']
x

,american_name,country,lat,long,date,conf_count,new_date
35,"Seattle, WA",US,47.6062,-122.3321,1/22/20,1,2020-01-22
36,"Chicago, IL",US,41.8781,-87.6298,1/22/20,0,2020-01-22
37,"Tempe, AZ",US,33.4255,-111.9400,1/22/20,0,2020-01-22
47,"Orange, CA",US,33.7879,-117.8531,1/22/20,0,2020-01-22
48,"Los Angeles, CA",US,34.0522,-118.2437,1/22/20,0,2020-01-22
...,...,...,...,...,...,...,...
4295,"Travis, CA (From Diamond Princess)",US,38.2721,-121.9399,2/28/20,0,2020-02-28
4297,"Lackland, TX (From Diamond Princess)",US,29.3829,-98.6134,2/28/20,0,2020-02-28
4299,"Humboldt County, CA",US,40.7450,-123.8695,2/28/20,1,2020-02-28
4300,"Sacramento County, CA",US,38.4747,-121.3542,2/28/20,2,2020-02-28


In [6]:
# Make sure country names in both files match
df_confirmed_long.loc[(df_confirmed_long.country == 'Mainland China'),'country']='China'
df_confirmed_long.loc[(df_confirmed_long.country == 'US'),'country']='United States of America'
df_confirmed_long = df_confirmed_long.set_index(['date', 'american_name'])

df_cured_long.loc[(df_cured_long.country == 'Mainland China'),'country']='China'
df_cured_long.loc[(df_cured_long.country == 'US'),'country']='United States of America'
df_cured_long = df_cured_long.set_index(['date', 'american_name'])

df_death_long.loc[(df_death_long.country == 'Mainland China'),'country']='China'
df_death_long.loc[(df_death_long.country == 'US'),'country']='United States of America'
df_death_long = df_death_long.set_index(['date', 'american_name'])


x = df_confirmed_long[df_confirmed_long['country'] == 'United States of America']
x

country  \
date    american_name                                                           
1/22/20 Seattle, WA                                  United States of America   
        Chicago, IL                                  United States of America   
        Tempe, AZ                                    United States of America   
        Orange, CA                                   United States of America   
        Los Angeles, CA                              United States of America   
...                                                                       ...   
2/28/20 Travis, CA (From Diamond Princess)           United States of America   
        Lackland, TX (From Diamond Princess)         United States of America   
        Humboldt County, CA                          United States of America   
        Sacramento County, CA                        United States of America   
        Unassigned Location (From Diamond Princess)  United States of America   

                                                         lat      long  \
date    american_name                                                    
1/22/20 Seattle, WA                                  47.6062 -122.3321   
        Chicago, IL                                  41.8781  -87.6298   
        Tempe, AZ                                    33.4255 -111.9400   
        Orange, CA                                   33.7879 -117.8531   
        Los Angeles, CA                              34.0522 -118.2437   
...                                                      ...       ...   
2/28/20 Travis, CA (From Diamond Princess)           38.2721 -121.9399   
        Lackland, TX (From Diamond Princess)         29.3829  -98.6134   
        Humboldt County, CA                          40.7450 -123.8695   
        Sacramento County, CA                        38.4747 -121.3542   
        Unassigned Location (From Diamond Princess)  35.4437  139.6380   

                                                     conf_count   new_date  
date    american_name                                                       
1/22/20 Seattle, WA                                           1 2020-01-22  
        Chicago, IL                                           0 2020-01-22  
        Tempe, AZ                                             0 2020-01-22  
        Orange, CA                                            0 2020-01-22  
        Los Angeles, CA                                       0 2020-01-22  
...                                                         ...        ...  
2/28/20 Travis, CA (From Diamond Princess)                    0 2020-02-28  
        Lackland, TX (From Diamond Princess)                  0 2020-02-28  
        Humboldt County, CA                                   1 2020-02-28  
        Sacramento County, CA                                 2 2020-02-28  
        Unassigned Location (From Diamond Princess)          44 2020-02-28  

[646 rows x 5 columns]

In [7]:
# Merge all 3 datasets
df_merged = pd.merge(df_confirmed_long, df_cured_long, how='left', on=['date', 'american_name', 'country','lat','long'])

df_merged = pd.merge(df_merged, df_death_long,how='left', on=['date', 'american_name', 'country','lat','long'])
df_merged = df_merged.reset_index()
df_merged = df_merged.drop(['date'], axis=1)
df_merged['country_2'] = df_merged['country']
df_merged = df_merged.rename(columns={"new_date":"date"})

output_path = os.path.join("static/world_data", "df_world_all.csv")
df_merged.to_csv(output_path)

df_merged
x = df_death_long[df_death_long['country'] == 'United States of America']
x

country  \
date    american_name                                                           
1/22/20 Seattle, WA                                  United States of America   
        Chicago, IL                                  United States of America   
        Tempe, AZ                                    United States of America   
        Orange, CA                                   United States of America   
        Los Angeles, CA                              United States of America   
...                                                                       ...   
2/28/20 Travis, CA (From Diamond Princess)           United States of America   
        Lackland, TX (From Diamond Princess)         United States of America   
        Humboldt County, CA                          United States of America   
        Sacramento County, CA                        United States of America   
        Unassigned Location (From Diamond Princess)  United States of America   

                                                         lat      long  \
date    american_name                                                    
1/22/20 Seattle, WA                                  47.6062 -122.3321   
        Chicago, IL                                  41.8781  -87.6298   
        Tempe, AZ                                    33.4255 -111.9400   
        Orange, CA                                   33.7879 -117.8531   
        Los Angeles, CA                              34.0522 -118.2437   
...                                                      ...       ...   
2/28/20 Travis, CA (From Diamond Princess)           38.2721 -121.9399   
        Lackland, TX (From Diamond Princess)         29.3829  -98.6134   
        Humboldt County, CA                          40.7450 -123.8695   
        Sacramento County, CA                        38.4747 -121.3542   
        Unassigned Location (From Diamond Princess)  35.4437  139.6380   

                                                     dead_count  
date    american_name                                            
1/22/20 Seattle, WA                                           0  
        Chicago, IL                                           0  
        Tempe, AZ                                             0  
        Orange, CA                                            0  
        Los Angeles, CA                                       0  
...                                                         ...  
2/28/20 Travis, CA (From Diamond Princess)                    0  
        Lackland, TX (From Diamond Princess)                  0  
        Humboldt County, CA                                   0  
        Sacramento County, CA                                 0  
        Unassigned Location (From Diamond Princess)           0  

[646 rows x 4 columns]

In [8]:
last_date = df_merged['date'].max()
print(last_date)

2020-02-28 00:00:00


In [9]:
current_date = datetime.now()
print("Current Date ",current_date)

# start_date = datetime.strptime('2020-01-22', "%Y-%m-%d")
start_date = last_date
print("Start Date", start_date)
# end_date = start_date + timedelta(days=10)
# print(end_date)

while current_date >= start_date:
    pass_date = start_date.strftime("%Y-%m-%d")
    print(pass_date)
    create_geojson(pass_date)
    start_date = start_date + timedelta(days=1)

# create_geojson('2020-02-26')

Current Date  2020-02-29 06:16:18.288417
Start Date 2020-02-28 00:00:00
2020-02-28
2020-02-29
